### Fourth Order Runge Kutta method $\mathtt{rk4}$ 
Obtains $O(h^4)$ precision by approximating $y$ as a taylor series up to $h^2$ (a parabola) at the midpoint of the interval, which leads to the cancellation of lower order error.
$\mathtt{rk4}$  requires the evaluation of four intermediate slopes 
\begin{equation}
\mathbf{y}_{n+1}=\mathbf{y}_{n}+\frac{1}{6}(\mathbf{k}_1+2\mathbf{k}_2+2\mathbf{k}_3+\mathbf{k}_4)
\end{equation}
and these are approximated sith the Euler algorithm to: 
\begin{align}
\mathbf{k}_1&=h\cdot \mathbf{f}(t_n,y_n)\\
\mathbf{k}_2&=h \cdot \mathbf{f}(t_n+\frac{h}{2},y_n+\frac{\mathbf{k}_1}{2})\\
\mathbf{k}_3&=h \cdot \mathbf{f}(t_n+\frac{h}{2},y_n+\frac{\mathbf{k}_2}{2})\\
\mathbf{k}_4&=h \cdot \mathbf{f}(t_n+h,y_n+\mathbf{k}_3)
\end{align}

In [76]:
import numpy as np
import vpython as vp

In [77]:
#Initialization
a = 0.
b = 10.
n = 100
ydumb = np.zeros((2), float); y = np.zeros((2), float)
fReturn = np.zeros((2), float); k1 = np.zeros((2), float)
k4 = np.zeros((2), float); k3 = np.zeros((2), float)
k4 = np.zeros((2), float)
y[0] = 3. ; y[1] = -5.
t = a ; h=(b-a)/n

#force function, the right hand side
def f(t,y):
    fReturn[0] = y[1]
    fReturn[1] = -100.*y[0] -2.*y[1] + 10.*np.sin(3.*t)
    return fReturn

In [78]:
scene = vp.canvas()
graph1 = vp.graph(x = 0, y = 0, width = 400, height = 400, title = 'RK4', xtitle = 't', 
                  ytitle = 'Y[0]=x(t)', xmin = 0, xmax =10, ymin = -2, ymax = 3)
funct1 = vp.gcurve(color = vp.color.yellow)
graph2 = vp.graph(x = 0, y = 0, width = 400, height = 400, title = 'RK4', xtitle = 't', 
                  ytitle = 'Y[1]=dx(t)/dt', xmin = 0, xmax =10, ymin = -25, ymax = 18)
funct2 = vp.gcurve(color = vp.color.red)

<IPython.core.display.Javascript object>

In [79]:
def rk4(t,h,n):
    k1 = [0]*n
    k2 = [0]*n
    k3 = [0]*n
    k4 = [0]*n
    fR = [0]*n
    ydumb = [0]*n
    fR = f(t,y)   #return right hand side
    for i in range(0,n):
        k1[i] = h*fR[i]
    for i in range(0,n):
        ydumb[i] =y[i] + k1[i]/2
    k2 = h*f(t+h/2., ydumb)
    for i in range(0,n):
        ydumb[i] =y[i] + k1[i]/2
    k3 = h*f(t+h/2., ydumb)
    for i in range(0,n):
        ydumb[i] = y[i] + k3[i]
    k4 = h*f(t+h, ydumb)
    for i in range(0,n):
        y[i] = y[i] + (k1[i] +2.*(k2[i]+k3[i])+k4[i])/6. 
    return y

In [80]:
while (t<b):              #time loop
    if ((t+h)>b):
        h = b-t             #last step
    y = rk4(t,h,2)
    t = t+h
    vp.rate(30)
    funct1.plot(pos = (t,y[0]))
    funct2.plot(pos = (t,y[1]))

### Adaptive step size Runge Kutta $\mathtt{rk45}$
Varies the step size while doing the integration with the hope of obtaining better precision and maybe better speed. It automatically doubles the step size and tests to see how an estimate of the error changes. If the error is still within acceptable bounds, the algorithm will continue to use the larger step size and thus speed up the computation; if the error is too large, the algorithm will decrease the step size until an acceptable error is found.


In [31]:
import numpy as np
import vpython as vp

In [32]:
a = 0.; b =10          # endpoints
Tol = 1.0E-8           # error tolerence
ydumb = np.zeros((2),float)  # initialization
y = np.zeros((2),float)
fReturn = np.zeros((2),float)
err = np.zeros((2),float)
k1 = np.zeros((2),float)
k2 = np.zeros((2),float)
k3 = np.zeros((2),float)
k4 = np.zeros((2),float)
k5 = np.zeros((2),float)
k6 = np.zeros((2),float)
n = 20
y[0] = 1. ; y[1] = 0.
h = (b-a)/n; t = a; j = 0
hmin = h/64; hmax = h*64     # Min and Max step sizes
flops = 0; Eexact = 0.; error = 0.
sum = 0.

In [33]:
def f(t, y, fReturn): # force function
    fReturn[0] = y[1]
    fReturn[1] = -6.*pow(y[0], 5.)

In [35]:
scene = vp.canvas()
graph1 = vp.graph(width = 600, height = 600, title = 'RK45', xtitle = 't', ytitle = 'Y[0]=x(t)')
funct1 = vp.gcurve(color = vp.color.blue)
graph2 = vp.graph(width = 600, height = 600, title = 'RK45', xtitle = 't', ytitle = 'Y[1]=dx(t)/dt')
funct2 = vp.gcurve(color = vp.color.red)
funct1.plot(pos = (t,y[0]))
funct2.plot(pos = (t,y[1]))

<IPython.core.display.Javascript object>

In [37]:
while (t < b) :              #loop over time
    funct1.plot(pos = (t, y[0]))
    funct2.plot(pos = (t, y[1]))
    
    if ((t+h)>b):
        h = b-t            #last step
        
    f(t, y, fReturn)       #evaluate f, return in fReturn          
    k1[0] = h*fReturn[0];   k1[1] = h*fReturn[1]   #calculate k1
    
    for i in range(0,2):     #calculate k2
        ydumb[i] = y[i] + k1[i]/4
    f(t+ h/4, ydumb, fReturn)
    k2[0] = h*fReturn[0];   k2[1] = h*fReturn[1]
    
    for i in range(0,2):     #calculate k3                                    
        ydumb[i] = y[i] +3 *k1[i]/32 + 9*k2[i]/32
    f(t+3*h/8, ydumb, fReturn)
    k3[0] = h*fReturn[0]; k3[1] = h*fReturn[1]
    
    for i in range(0,2):   #calculate k4
        ydumb[i] = y[i] + 1932*k1[i]/2197 - 7200*k2[i]/2197. + 7296*k3[i]/2197
    f(t+12*h/13, ydumb, fReturn)
    k4[0] = h*fReturn[0]; k4[1] = h*fReturn[1]
    
    for i in range(0,2):   #calculate k5
        ydumb[i] = y[i] + 439*k1[i]/216 - 8*k2[i] + 3680*k3[i]/513 - 845*k4[i]/4104
    f(t+h, ydumb, fReturn)
    k5[0] = h*fReturn[0]; k5[1] = h*fReturn[1]
    
    for i in range(0,2):  #calculaze k6
        ydumb[i] = y[i] - 8*k1[i]/27 + 2*k2[i] - 3544*k3[i]/2565 + 1859*k4[i]/4104 - 11*k5[i]/40
    f(t+h/2, ydumb, fReturn)
    k6[0] = h*fReturn[0]; k6[1] = h*fReturn[1]
    
    for i in range(0,2):   #calculate error
        err[i] = abs(k1[i]/360 - 128*k3[i]/4275 - 2197*k4[i]/75240 + k5[i]/50. * 2*k6[i]/55)
    
    if (err[0] < Tol or err[1] < Tol or h <= 2*hmin):  #accept step
        for i in range(0,2):
            y[i] = y[i] + 25*k1[i]/216. + 1408*k3[i]/2565. + 2197*k4[i]/4104. - k5[i]/5.
        t = t + h
        j = j + 1
        
    if (err[0] == 0 or err[1]  == 0):  #trap division by 0
        s = 0
    else:
        s = 0.84*pow(Tol*h/err[0], 0.25)  
        
    if (s < 0.75 and h > 2*hmin):
        h /= 2.              #reduce step
    else:
        if (s > 1.5 and 2* h < hmax):
            h *= 2.         #increase step
            
    flops = flops + 1
    E = pow(y[0], 6.) + 0.5 * y[1] * y[1]
    Eexact = 1.
    error = abs((E - Eexact)/Eexact)
    sum += error
print("<error>= ", sum/flops, ", flops = ", flops)

<error>=  1.0318188164588248e-07 , flops =  645


### Adams-Bashforth-Moulton predictor-corrector scheme
Uses the solution from previous steps $y_{n-2}$ and $y_{n-1}$ in addition to $y_n$ to predict $y_{n+1}$. Many of these types of methods tend to be like a Newton's search; we start with a guess or prediction for the next step and then use an algorithm such as $\mathtt{rk4}$ to check the prediction and thereby obtain a correction. As with $\mathtt{rk45}$, one can use the correction as a measure of the error and then adjust the step size to obtain improved precision.


In [38]:
# Adams BM method to integrate ODe
# Solves y' = (t - y)/2, with y[0] = 1 over [0,3]
import numpy as np
import vpython as vp

In [61]:
scene = vp.canvas()
numgr = vp.graph(x = 0, y = 0, width = 600, height = 300, xmin = 0.0, xmax = 3.0,
                title = "Numerical Solution", xtitle = 't', 
                ytitle = 'y', ymax = 2., ymin = 0.5)
numsol = vp.gcurve(color = vp.color.red)
exactgr = vp.graph(x = 0, y = 300, width = 600, height = 300, title = "Exact solution", 
                   xtitle = 't', ytitle = 'y', xmax = 3.0, xmin = 0.0,
                   ymax = 2.0, ymin = 0.5)
exsol = vp.gcurve(color = vp.color.blue)

<IPython.core.display.Javascript object>

In [62]:
n = 24  # N steps > 3
A = 0; B = 3.
t = [0]*500; y = [0]*500; yy = [0]*4

In [63]:
def f(t, y):      #right hand side function
    return (t -y)/2.0

In [64]:
def rk4(t, yy, h1): # Runge-Kutta
    for i in range(0,3):
        t = h1 * i
        k0 = h1 * f(t, y[i])
        k1 = h1 * f(t + h1/2., yy[i] + k0/2.)
        k2 = h1 * f(t + h1/2., yy[i] + k1/2.)
        k3 = h1 * f(t + h1, yy[i] + k2)
        yy[i + 1] = yy[i] + (1./6.) * (k0 + 2.*k1 + 2.*k2 + k3)
        #print(i,yy[i])
    return yy[3]        

In [65]:
def ABM(a,b,N):
    #compute three additional starting values using rk
    h = (b - a) / N  #step
    t[0] = a; y[0] = 1.00; F0 = f(t[0], y[0])
    
    for k in range(1,4): #compute steps
        t[k] = a + k * h
        
    y[1] = rk4(t[1], y, h)      # 1st step
    y[2] = rk4(t[2], y, h)      # 2nd step
    y[3] = rk4(t[3], y, h)      # 3rd step
    
    F1 = f(t[1], y[1])
    F2 = f(t[2], y[2])
    F3 = f(t[3], y[3])
    h2 = h/24.
    
    for k in range(3,N):   
        p = y[k] + h2 * (-9.*F0 + 37. * F1 - 59. * F2 + 55. * F3)     #predictor
        t[k+1] = a + h*(k+1)   # next abscissa
        F4 = f(t[k+1], p)
        y[k+1] = y[k] + h2 * (F1 - 5. * F2 + 19. * F3 + 9. * F4)      #corrector
        F0 = F1                    #y_n-2
        F1 = F2                    #y_n-1
        F2 = F3                    #y_n
        F3 = f(t[k + 1], y[k + 1]) #y_n+1
    return t,y

In [66]:
print(" k   t       Y numerical      Y exact")
t, y = ABM(A,b,n)
for k in range(0, n+1):
    print(" %3d %5.3f %12.11f %12.11f"%(k,t[k],y[k],(3.*np.exp(-t[k]/2.)-2.+t[k])))
    numsol.plot(pos = (t[k], y[k]))
    exsol.plot(pos = (t[k], 3.*np.exp(-t[k]/2.) -2. + t[k]))

 k   t       Y numerical      Y exact
   0 0.000 1.00000000000 1.00000000000
   1 0.417 0.85248518579 0.85247570512
   2 0.833 0.81107061936 0.81105522393
   3 1.250 0.85580303576 0.85578428556
   4 1.667 0.97043713240 0.97046129219
   5 2.083 1.14188077086 1.14193157771
   6 2.500 1.35944828773 1.35951439058
   7 2.917 1.61446344116 1.61453764042
   8 3.333 1.89988323679 1.89996014185
   9 3.750 2.20998893937 2.21006490053
  10 4.167 2.54013742615 2.54021008100
  11 4.583 2.88655890304 2.88662680829
  12 5.000 3.24619262371 3.24625499587
  13 5.417 3.61655352984 3.61661004824
  14 5.833 3.99562397172 3.99567463200
  15 6.250 4.38176579436 4.38181080087
  16 6.667 4.77364895928 4.77368864671
  17 7.083 5.17019359163 5.17022836877
  18 7.500 5.57052292755 5.57055323757
  19 7.917 5.97392511152 5.97395140459
  20 8.333 6.37982217918 6.37984489413
  21 8.750 6.78774487403 6.78776442673
  22 9.167 7.19731220087 7.19732897732
  23 9.583 7.60821482495 7.60822917781
  24 10.000 8.02020159351 

### Assessment: $\mathtt{rk2}$ vs. $\mathtt{rk4}$ vs. $\mathtt{rk45}$
#### Harmonic oscillations
1. Write your own $\mathtt{rk2}$ method for general ODE by making the derivative function $f(x,t)$ a seperate method
2. Harmonic oscillators are \textbf{isochronous}: The period does not change as the amplitude varies (the initial position).

In [284]:
import numpy as np
import vpython as vp

In [289]:
#Initialization
a = 0.
b = 3.
n = 100
ydumb = np.zeros((2), float); y = np.zeros((2), float)
fReturn = np.zeros((2), float); k1 = np.zeros((2), float)
k2 = np.zeros((2), float)
x_init = y[0] = 3. #initial position
v_init= y[1] = -5. #initial velocity
t = a ; h=(b-a)/n
A = 3.1
m = 10.
k = 4* m* np.pi**2
omega = np.sqrt(k/m)
p = 2  #to obtain harmonic oscillation v(x) = 1/p* k* x^p
alpha = 0.
fAnalytic = np.zeros((2), float)
c = np.arcsin(x_init/A)
d = np.arccos(v_init/(omega*A))
#force function, the right hand side
def f(t,y):
    #f = f_ext+f_k
    #f_ext :exerced by the hand
    #f_k = restoring force of the spring
    fReturn[0] = y[1]
    fReturn[1] = -k/m*pow(y[0], p-1) 
    #fReturn[1] = 1/m*(A*np.sin(omega*t)- k*pow(y[0], p-1)) 
    #fReturn[1] = 1/m*(- k*y[0]*(1-alpha*y[0])) 
    return fReturn
#analytic function
def fanalytic(t):
    fAnalytic[0] = A * np.sin(omega * t + c)
    fAnalytic[1] = omega * A * np.cos(omega * t + d)
    return fAnalytic

In [290]:
def rk2(t,h,n): 
    # n: order of the ode
    # h: step length
    # t: time step
    k1 = [0]*n
    k2 = [0]*n
    ydumb = [0]*n
    #fR = [0]*n
    fR = f(t,y)
    for i in range(0,n):
        k1[i] = h*fR[i]
    for i in range(0,n):
        ydumb[i] = y[i]+k1[i]/2.
    k2 = h * f(t+h/2., ydumb)
    for i in range(0,n):
        y[i] = y[i] + k2[i]
    return y

In [291]:
scene = vp.canvas()
graph1 = vp.graph(x = 0, y = 0, width = 400, height = 400, title = 'RK2', xtitle = 't', 
                  ytitle = 'Y[0]=x(t)', xmin = 0, xmax =3)
funct1 = vp.gcurve(color = vp.color.yellow)
analytic1 = vp.gcurve(color = vp.color.green)
graph2 = vp.graph(x = 0, y = 0, width = 400, height = 400, title = 'RK2', xtitle = 't', 
                  ytitle = 'Y[1]=dx(t)/dt', xmin = 0, xmax =3)
funct2 = vp.gcurve(color = vp.color.red)
analytic2 = vp.gcurve(color= vp.color.green)

<IPython.core.display.Javascript object>

In [292]:
while (t<b):              #time loop
    if ((t+h)>b):
        h = b-t             #last step
    if t==0:
        funct1.plot(pos = (t,x_init))
        funct2.plot(pos = (t,v_init))
    yanalytic = fanalytic(t)
    analytic1.plot(pos = (t,yanalytic[0]))
    analytic2.plot(pos = (t,yanalytic[1]))
    y = rk2(t,h,2)
    t = t+h
    funct1.plot(pos = (t,y[0]))
    funct2.plot(pos = (t,y[1]))
    vp.rate(10)

#### Anharmonic oscillation
1. Anharmonic oscillators are \textbf{nonisochnonous}, vibrations with different amplitudes have different periods. 

In [355]:
import numpy as np
import vpython as vp

In [369]:
#Initialization
a = 0.
b = 3.
n = 100
ydumb = np.zeros((2), float); y = np.zeros((2), float)
fReturn = np.zeros((2), float); k1 = np.zeros((2), float)
k2 = np.zeros((2), float)
x_init = y[0] = 20. #initial position
v_init= y[1] = -5. #initial velocity
t = a ; h=(b-a)/n
m = 10.
k = 5.
p = 4 #to obtain anharmonic oscillation , p>=2
#force function, the right hand side
def f(t,y):
    #f = f_ext+f_k
    #f_ext :exerced by the hand
    #f_k = restoring force of the spring
    fReturn[0] = y[1]
    fReturn[1] = -k/m*pow(y[0], p-1) 
    return fReturn

In [370]:
scene = vp.canvas()
graph1 = vp.graph(x = 0, y = 0, width = 400, height = 400, title = 'RK2', xtitle = 't', 
                  ytitle = 'Y[0]=x(t)', xmin = 0, xmax =3)
funct1 = vp.gcurve(color = vp.color.yellow)
graph2 = vp.graph(x = 0, y = 0, width = 400, height = 400, title = 'RK2', xtitle = 't', 
                  ytitle = 'Y[1]=dx(t)/dt', xmin = 0, xmax =3)
funct2 = vp.gcurve(color = vp.color.red)

<IPython.core.display.Javascript object>

In [371]:
def rk4(t,h,n):
    k1 = [0]*n
    k2 = [0]*n
    k3 = [0]*n
    k4 = [0]*n
    fR = [0]*n
    ydumb = [0]*n
    fR = f(t,y)   #return right hand side
    for i in range(0,n):
        k1[i] = h*fR[i]
    for i in range(0,n):
        ydumb[i] =y[i] + k1[i]/2
    k2 = h*f(t+h/2., ydumb)
    for i in range(0,n):
        ydumb[i] =y[i] + k1[i]/2
    k3 = h*f(t+h/2., ydumb)
    for i in range(0,n):
        ydumb[i] = y[i] + k3[i]
    k4 = h*f(t+h, ydumb)
    for i in range(0,n):
        y[i] = y[i] + (k1[i] +2.*(k2[i]+k3[i])+k4[i])/6. 
    return y

In [372]:
while (t<b):              #time loop
    if ((t+h)>b):
        h = b-t             #last step
    y = rk4(t,h,2)
    t = t+h
    vp.rate(30)
    funct1.plot(pos = (t,y[0]))
    funct2.plot(pos = (t,y[1]))

2. Explain why the shapes of the oscillations change for different $p$'s?
Remember, the oscillator has the form $F_k=-kx^{p-1}$

#### Energy conservation
Plot the potential energy $PE(t)=V[x(t)]$, the kinetic energy $KE(t)=mv^2(t)/2$ and the total energy $E(t)=KE(t)+PE(t)$ for $50$ periods. Comment on the correlation between $PE(t)$ and $KE(t)$ and how it depends on the potential's parameters. 

In [437]:
import numpy as np
import vpython as vp

In [456]:
#Initialization
a = 0.
b =10.
n = 1000
ydumb = np.zeros((2), float); y = np.zeros((2), float)
fReturn = np.zeros((2), float); k1 = np.zeros((2), float)
k2 = np.zeros((2), float)
x_init = y[0] = 20. #initial position
v_init= y[1] = -5. #initial velocity
t = a ; h=(b-a)/n
m = 10.
k = 2.
p = 2 #to obtain anharmonic oscillation , p>=2
#force function, the right hand side
def f(t,y):
    #f = f_ext+f_k
    #f_ext :exerced by the hand
    #f_k = restoring force of the spring
    fReturn[0] = y[1]
    fReturn[1] = -k/m*pow(y[0], p-1) 
    return fReturn

In [457]:
def rk4(t,h,n):
    k1 = [0]*n
    k2 = [0]*n
    k3 = [0]*n
    k4 = [0]*n
    fR = [0]*n
    ydumb = [0]*n
    fR = f(t,y)   #return right hand side
    for i in range(0,n):
        k1[i] = h*fR[i]
    for i in range(0,n):
        ydumb[i] =y[i] + k1[i]/2
    k2 = h*f(t+h/2., ydumb)
    for i in range(0,n):
        ydumb[i] =y[i] + k1[i]/2
    k3 = h*f(t+h/2., ydumb)
    for i in range(0,n):
        ydumb[i] = y[i] + k3[i]
    k4 = h*f(t+h, ydumb)
    for i in range(0,n):
        y[i] = y[i] + (k1[i] +2.*(k2[i]+k3[i])+k4[i])/6. 
    return y

In [447]:
scene = vp.canvas()
potential_energy_graph = vp.graph(width = 500, height = 500, title = 'potential Energy plot', xtitle ='t',
                       ytitle = 'potential Energy')
potential_energy_plot = vp.gcurve(color = vp.color.red)
kinetic_energy_graph = vp.graph(width = 500, height = 500, title = 'Kinetic Energy plot', xtitle ='t',
                       ytitle = 'Kinetic Energy')
kinetic_energy_plot = vp.gcurve(color = vp.color.magenta)
total_energy_graph = vp.graph(width = 500, height = 500, title = 'Total Energy plot', xtitle ='t',
                       ytitle = 'total Energy')
total_energy_plot = vp.gcurve(color = vp.color.green)

<IPython.core.display.Javascript object>

In [444]:
while (t<b):              #time loop
    if ((t+h)>b):
        h = b-t             #last step
    y = rk4(t,h,2)
    t = t+h
    potential_energy = 1/p * k *pow(y[0],p)
    kinetic_energy = m * (y[1]**2)/2
    total_energy = potential_energy + kinetic_energy
    vp.rate(30)
    potential_energy_plot.plot(pos =(t, potential_energy))
    kinetic_energy_plot.plot(pos =(t, kinetic_energy))
    total_energy_plot.plot(pos =(t, total_energy))

Check the long term stabilitay of your solution by plotting
$-\ln_{10}\|\frac{E(t)-E(t=0)}{E(t=0)}\|\approx$ number of places of precision.


In [448]:
stability_graph = vp.graph(width = 500, height = 500, title = 'stabiltiy plot', 
                           xtitle ='t',
                           ytitle = '-log|relative error|')
stability_plot = vp.gcurve(color = vp.color.magenta)                          
E0 = 1/p * k *pow(x_init,p) + m * (v_init**2)/2
while (t<b):              #time loop
    if ((t+h)>b):
        h = b-t             #last step
    y = rk4(t,h,2)
    t = t+h
    potential_energy = 1/p * k *pow(y[0],p)
    kinetic_energy = m * (y[1]**2)/2
    total_energy = potential_energy + kinetic_energy
    vp.rate(30)
    rel_error = -np.log(abs((total_energy-E0)/E0))
    stability_plot.plot(pos = (t, rel_error))

Because a particle bound by a large $p$-oscillator is essentially 'free' most of the time, you should observe that the average of its kinetic energy over time exceeds its average potential energy. This is actually the physics behind the Virial theorem for a power-law potential.
 \begin{equation}
 \langle KE\rangle =\frac{p}{2}\langle PE\rangle
 \end{equation}
 Verify that your solution satisfies the Virial theorem.

In [504]:
a = 0.
b =10.
n = 100000
ydumb = np.zeros((2), float); y = np.zeros((2), float)
fReturn = np.zeros((2), float); k1 = np.zeros((2), float)
k2 = np.zeros((2), float)
x_init = y[0] = 20. #initial position
v_init= y[1] = -5. #initial velocity
t = a ; h=(b-a)/n
m = 10.
k = 2.
p = 6.

In [505]:
sum_potential_energy =0
sum_kinetic_energy = 0
while (t<b):              #time loop
    if ((t+h)>b):
        h = b-t             #last step
    y = rk4(t,h,2)
    t = t+h
    sum_potential_energy += 1/p * k *pow(y[0],p)
    sum_kinetic_energy += m * (y[1]**2)/2
average_PE = 1/n * sum_potential_energy
average_KE = 1/n * sum_kinetic_energy
print('Virial theorem almost satisfied')
print ('<KE>        p/2*<PE>')
print('%6.2f    %6.2f'% (average_KE, p/2* average_PE))

Virial theorem almost satisfied
<KE>        p/2*<PE>
16022498.92    16022029.51


### Nonlinear Resonances, Beats, Friction
#### Friction